In [ ]:
import pandas as pd
import datasets
import tqdm, os
from collections.abc import Iterable
from langchain_core.documents.base import Document

from datetime import datetime
from copy import deepcopy

from dotenv import load_dotenv
load_dotenv()

huggingface_path = 'ErzhuoShao/SciSciGPT-SciSciNet'
revision = 'ef5553f34410575c8cab8ad209a7b11a4253b2b6'

In [ ]:
from datasets import load_dataset
fields = load_dataset(
	huggingface_path, revision=revision, split="train", 
	data_files="fields.parquet"
).to_pandas()

institutions = load_dataset(
	huggingface_path, revision=revision, split="train", 
	data_files="institutions.parquet"
).to_pandas()

In [ ]:
from pinecone import Pinecone
pc = Pinecone()
index = pc.Index(os.getenv("NAME_SEARCH_INDEX"))

from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [ ]:
names = institutions["institution_name"].tolist()
embedding_list = []
for i in tqdm.trange(0, len(names), 128):
    embedding_list += embeddings.embed_documents(names[i:i+128])
institutions["embedding"] = embedding_list

for institution in tqdm.tqdm(institutions.to_dict(orient="records")):
    institution = deepcopy(institution)
    
    embedding = institution.pop("embedding")
    id = str(institution["institution_id"])

    institution = {k: v for k, v in institution.items() if v != None}
    index.upsert(
        vectors=[ { "id": id, "values": embedding, "metadata": institution } ],
        namespace="institution_name"
    )

In [ ]:
names = fields["field_name"].tolist()
embedding_list = []
for i in tqdm.trange(0, len(names), 128):
    embedding_list += embeddings.embed_documents(names[i:i+128])
fields["embedding"] = embedding_list

for field in tqdm.tqdm(fields.to_dict(orient="records")):
    field = deepcopy(field)
    embedding = field.pop("embedding")
    id = str(field["field_id"])
	
    field = {k: v for k, v in field.items() if v != None}
    index.upsert(
        vectors=[ { "id": id, "values": embedding, "metadata": field } ],
        namespace="field_name"
    )